# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math 

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('../sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'AG/LGANLSDM(OESESETRA N  AKL)QTBC',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'ioclffia',
 'close': None,
 'closeTime': None,
 'closeSource': 'fifolica',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 132.43,
 'latestSource': 'IEX real time price',
 'latestTime': '2:48:01 PM',
 'latestUpdate': 1611889828789,
 'latestVolume': None,
 'iexRealtimePrice': 129.06,
 'iexRealtimeSize': 101,
 'iexLastUpdated': 1638006091984,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 130.51,
 'previousVolume': 113031962,
 'change': 0.64,
 'changePercent': 0.0049,
 'volume': None,
 'iexMarketPercent': 0.013965611825637759,
 'iexVolume': 9120

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

39.22

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,128.361,56.52,N/A
1,AAL,16.407,-1.19,N/A
2,AAP,170.100,25.4,N/A
3,AAPL,133.350,39.53,N/A
4,ABBV,112.350,25.36,N/A
...,...,...,...,...
500,YUM,109.340,32.24,N/A
501,ZBH,165.700,-225.4,N/A
502,ZBRA,410.940,47.83,N/A
503,ZION,50.610,20.83,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,43.200,2.64,N/A
1,NLOK,21.600,4.08,N/A
2,AIV,4.980,4.79,N/A
3,BIO,621.790,5.47,N/A
4,UNM,26.450,5.83,N/A
5,ALL,111.600,7.77,N/A
6,AFL,48.270,7.78,N/A
7,CPB,46.710,7.99,N/A
8,EBAY,57.840,8.11,N/A
9,KIM,17.027,8.28,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

You can now use the global portfolio_size variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,43.200,2.64,4629
1,NLOK,21.600,4.08,9259
2,AIV,4.980,4.79,40160
3,BIO,621.790,5.47,321
4,UNM,26.450,5.83,7561
5,ALL,111.600,7.77,1792
6,AFL,48.270,7.78,4143
7,CPB,46.710,7.99,4281
8,EBAY,57.840,8.11,3457
9,KIM,17.027,8.28,11746


In [10]:
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,129.468,N/A,55.19,N/A,8.15,N/A,7.58,N/A,33.468288,N/A,13.915912,N/A,N/A
1,AAL,16.100,N/A,-1.17,N/A,-1.74,N/A,0.3959,N/A,-5.897345,N/A,1.393807,N/A,N/A
2,AAP,173.500,N/A,24.3,N/A,3.08,N/A,1.17,N/A,10.987641,N/A,2.537277,N/A,N/A
3,AAPL,130.000,N/A,40.46,N/A,33.01,N/A,8.1,N/A,28.080108,N/A,20.312757,N/A,N/A
4,ABBV,116.150,N/A,25.35,N/A,13.08,N/A,5,N/A,16.768659,N/A,9.828063,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.410,N/A,32.25,N/A,-4.18,N/A,5.87,N/A,23.295913,N/A,9.922201,N/A,N/A
501,ZBH,160.780,N/A,-221.9,N/A,2.8,N/A,4.8,N/A,27.631277,N/A,8.028337,N/A,N/A
502,ZBRA,424.760,N/A,47.95,N/A,11.6,N/A,5.23,N/A,32.506365,N/A,11.849368,N/A,N/A
503,ZION,50.440,N/A,21.59,N/A,1.13,N/A,2.71,N/A,12.298135,N/A,3.119236,N/A,N/A


{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,218.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,238.420,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.88,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,33.210,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.830,N/A,14.43,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,53.700,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,30.190,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,160.150,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1855.790,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.590,N/A,31.52,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,129.468,N/A,55.19,N/A,8.15,N/A,7.5800,N/A,33.468288,N/A,13.915912,N/A,N/A
1,AAL,16.100,N/A,-1.17,N/A,-1.74,N/A,0.3959,N/A,-5.897345,N/A,1.393807,N/A,N/A
2,AAP,173.500,N/A,24.30,N/A,3.08,N/A,1.1700,N/A,10.987641,N/A,2.537277,N/A,N/A
3,AAPL,130.000,N/A,40.46,N/A,33.01,N/A,8.1000,N/A,28.080108,N/A,20.312757,N/A,N/A
4,ABBV,116.150,N/A,25.35,N/A,13.08,N/A,5.0000,N/A,16.768659,N/A,9.828063,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.410,N/A,32.25,N/A,-4.18,N/A,5.8700,N/A,23.295913,N/A,9.922201,N/A,N/A
501,ZBH,160.780,N/A,-221.90,N/A,2.80,N/A,4.8000,N/A,27.631277,N/A,8.028337,N/A,N/A
502,ZBRA,424.760,N/A,47.95,N/A,11.60,N/A,5.2300,N/A,32.506365,N/A,11.849368,N/A,N/A
503,ZION,50.440,N/A,21.59,N/A,1.13,N/A,2.7100,N/A,12.298135,N/A,3.119236,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.865347
1       0.154455
2        0.49505
3       0.776238
4       0.510891
         ...    
500     0.653465
501    0.0257426
502     0.821782
503     0.447525
504     0.827723
Name: PE Percentile, Length: 505, dtype: object
0       0.744554
1      0.0475248
2       0.411881
3       0.960396
4       0.855446
         ...    
500    0.0435644
501     0.386139
502     0.839604
503     0.106931
504     0.926733
Name: PB Percentile, Length: 505, dtype: object
0       0.816832
1      0.0237624
2       0.163366
3       0.843564
4        0.69505
         ...    
500     0.744554
501     0.683168
502     0.714851
503     0.422772
504     0.936634
Name: PS Percentile, Length: 505, dtype: object
0      0.847525
1      0.029703
2      0.241584
3       0.79802
4       0.49703
         ...   
500     0.70495
501    0.790099
502    0.843564
503    0.312871
504    0.849505
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.815842
1      0.0514851
2       0.114851
3       0.934

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,129.468,N/A,55.19,0.865347,8.15,0.744554,7.5800,0.816832,33.468288,0.847525,13.915912,0.815842,N/A
1,AAL,16.100,N/A,-1.17,0.154455,-1.74,0.0475248,0.3959,0.0237624,-5.897345,0.029703,1.393807,0.0514851,N/A
2,AAP,173.500,N/A,24.30,0.49505,3.08,0.411881,1.1700,0.163366,10.987641,0.241584,2.537277,0.114851,N/A
3,AAPL,130.000,N/A,40.46,0.776238,33.01,0.960396,8.1000,0.843564,28.080108,0.79802,20.312757,0.934653,N/A
4,ABBV,116.150,N/A,25.35,0.510891,13.08,0.855446,5.0000,0.69505,16.768659,0.49703,9.828063,0.649505,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.410,N/A,32.25,0.653465,-4.18,0.0435644,5.8700,0.744554,23.295913,0.70495,9.922201,0.655446,N/A
501,ZBH,160.780,N/A,-221.90,0.0257426,2.80,0.386139,4.8000,0.683168,27.631277,0.790099,8.028337,0.518812,N/A
502,ZBRA,424.760,N/A,47.95,0.821782,11.60,0.839604,5.2300,0.714851,32.506365,0.843564,11.849368,0.736634,N/A
503,ZION,50.440,N/A,21.59,0.447525,1.13,0.106931,2.7100,0.422772,12.298135,0.312871,3.119236,0.158416,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,129.468,N/A,55.19,0.865347,8.15,0.744554,7.5800,0.816832,33.468288,0.847525,13.915912,0.815842,0.81802
1,AAL,16.100,N/A,-1.17,0.154455,-1.74,0.0475248,0.3959,0.0237624,-5.897345,0.029703,1.393807,0.0514851,0.0613861
2,AAP,173.500,N/A,24.30,0.49505,3.08,0.411881,1.1700,0.163366,10.987641,0.241584,2.537277,0.114851,0.285347
3,AAPL,130.000,N/A,40.46,0.776238,33.01,0.960396,8.1000,0.843564,28.080108,0.79802,20.312757,0.934653,0.862574
4,ABBV,116.150,N/A,25.35,0.510891,13.08,0.855446,5.0000,0.69505,16.768659,0.49703,9.828063,0.649505,0.641584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.410,N/A,32.25,0.653465,-4.18,0.0435644,5.8700,0.744554,23.295913,0.70495,9.922201,0.655446,0.560396
501,ZBH,160.780,N/A,-221.90,0.0257426,2.80,0.386139,4.8000,0.683168,27.631277,0.790099,8.028337,0.518812,0.480792
502,ZBRA,424.760,N/A,47.95,0.821782,11.60,0.839604,5.2300,0.714851,32.506365,0.843564,11.849368,0.736634,0.791287
503,ZION,50.440,N/A,21.59,0.447525,1.13,0.106931,2.7100,0.422772,12.298135,0.312871,3.119236,0.158416,0.289703


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

In [20]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,F,10.519,19013,-251.93,0.0237624,1.2000,0.122772,0.3042,0.0118812,3.954013,0.039604,2.065085,0.0910891,0.0578218
1,AIG,42.750,4678,-7.48,0.112871,0.5868,0.0574257,0.7950,0.0712871,4.619535,0.0435644,0.779536,0.0158416,0.060198
2,AAL,16.100,12422,-1.17,0.154455,-1.7400,0.0475248,0.3959,0.0237624,-5.897345,0.029703,1.393807,0.0514851,0.0613861
3,UNM,27.110,7377,5.80,0.19604,0.4874,0.0534653,0.4483,0.0336634,2.772769,0.0356436,0.431134,0.0039604,0.0645545
4,MET,52.437,3814,8.91,0.207921,0.6427,0.0613861,0.7126,0.0594059,4.940103,0.049505,0.714511,0.0118812,0.0780198
5,BA,212.278,942,-26.79,0.0752475,-9.9900,0.0336634,2.0000,0.323762,-27.501902,0.0178218,-200.606418,0.0019802,0.090495
6,ALL,110.680,1807,7.99,0.2,1.3000,0.150495,0.7399,0.0653465,2.522485,0.0336634,0.742231,0.0138614,0.0926733
7,HIG,53.956,3706,11.41,0.233663,1.0800,0.10099,0.9324,0.0950495,4.616109,0.0415842,0.897064,0.0217822,0.0986139
8,FTI,11.867,16853,-0.94,0.182178,1.3100,0.157426,0.4086,0.0277228,3.673550,0.0376238,2.378060,0.110891,0.103168
9,ABC,108.920,1836,-6.29,0.120792,-22.2700,0.019802,0.1153,0.0039604,9.030324,0.140594,4.240841,0.233663,0.103762


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [24]:
writer.save()